In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.linalg as la

def get_num_vertices(adj_mat):
    return adj_mat.shape[0]

def get_num_edges(adj_mat):
    n = get_num_vertices(adj_mat)
    upper_tri_idx = np.triu_indices(n, 1)
    return np.count_nonzero(adj_mat[upper_tri_idx])

def compute_graph_volume(adj_mat):
    return float(np.sum(adj_mat))/2.0

def compute_graph_density(adj_mat):
    n = get_num_vertices(adj_mat)
    m = get_num_edges(adj_mat)
    
    density = float(2 * m)/float(n * (n - 1))
    return density

def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

def compute_degree_distribution(adj_mat):
    # Assuming a simple, possibly weighted graph, degree of each vertex
    # will be the sum along one axis
    deg_arr = np.sum(adj_mat, axis=0)
    return deg_arr

def compute_average_degree(adj_mat):
    deg_arr = compute_degree_distribution(adj_mat)
    n = get_num_vertices(adj_mat)
    
    avg_deg = float(sum(deg_arr))/float(n)
    return avg_deg

def display_degree_distribution(adj_mat, dataset_name, override_min, num_bins_log_log=50):
    deg_arr = compute_degree_distribution(adj_mat)
    #deg_arr = reject_outliers(deg_arr)
    # Display histogram of degree distribution
    plt.subplot(2, 1, 1)
    plt.hist(deg_arr, bins='auto')
    plt.title('Deg Dist of {}'.format(dataset_name))
    
    # Display histogram of degree distribution with log-log plot
    plt.subplot(2, 1, 2)
    max_val = np.amax(adj_mat)
    min_val = np.amin(adj_mat[adj_mat > override_min])
    print min_val
    pow_ten_min = 10 ** (np.floor(np.log10(min_val+0.1)))
    pow_ten_max = 10 ** (np.ceil(np.log10(max_val)))
    plt.hist(deg_arr, bins=np.logspace(np.log10(pow_ten_min), np.log10(pow_ten_max), num_bins_log_log))
    plt.gca().set_xscale('log')
    plt.title('Log-Log Deg Dist of {}'.format(dataset_name))
    
    plt.show()
    
def get_neighborhood_set_of_node(adj_mat, node):
    nbrhd = []
    n = get_num_vertices(adj_mat)
    
    # Find all edges connecting node to other nodes
    for j in range(0, n):
        if adj_mat[node][j] != 0:
            nbrhd.append(j)
    return nbrhd
    
def compute_clustering_coefficient(adj_mat, node):
    # Get the degree of the node
    node_deg = compute_degree_distribution(adj_mat)[node]
    
    if node_deg <= 1:
        #print("Node {} has degree {}. CC is undefined".format(node, node_deg))
        return 0
    
    # Get the neighborhood set of the node
    nbrhd = get_neighborhood_set_of_node(adj_mat, node)
       
    # Get reduced neighborhood matrix of node
    nbrhd_matrix = adj_mat[:, nbrhd]
    nbrhd_matrix = nbrhd_matrix[nbrhd, :]
       
    # Count the number of triangles formed by the neighborhood set
    num_triangles = np.sum(nbrhd_matrix)

    # Compute the clustering coefficient and add it to the array
    clustering_coefficient = float(2 * num_triangles)/float(node_deg * (node_deg - 1))
    return clustering_coefficient
    
def compute_all_clustering_coefficients(adj_mat):
    # For each node in the graph, compute the clustering coefficient and store
    # it in an array
    n = get_num_vertices(adj_mat)
    cluster_coeff_arr = np.zeros((n, 1))
    
    for node in range(0, n):
        # Compute the clustering coefficient and add it to the array
        cluster_coeff_arr[node] = compute_clustering_coefficient(adj_mat, node)
        
    return cluster_coeff_arr    

def display_clustering_coefficient_dist(adj_mat, dataset_name, num_bins_log_log=50):
    cluster_coeff_arr = compute_all_clustering_coefficients(adj_mat)
    #cluster_coeff_arr = reject_outliers(cluster_coeff_arr)
    # Display histogram of clustering coefficients
    plt.subplot(2, 1, 1)
    plt.hist(cluster_coeff_arr, bins='auto')
    plt.title('CC Dist of {}'.format(dataset_name))
    
    # Display histogram of degree distribution with log-log plot
    plt.subplot(2, 1, 2)
    max_val = np.amax(cluster_coeff_arr)
    min_val = np.amin(cluster_coeff_arr)
    pow_ten_min = 10 ** (np.floor(np.log10(min_val+0.1)))
    pow_ten_max = 10 ** (np.ceil(np.log10(max_val)))
    plt.hist(deg_arr, bins=np.logspace(np.log10(pow_ten_min), np.log10(pow_ten_max), num_bins_log_log))
    plt.gca().set_xscale('log')
    plt.title('Log-Log CC Dist of {}'.format(dataset_name))
    
    plt.show()

def compute_average_clustering_coefficient(adj_mat):
    n = get_num_vertices(adj_mat)
    avg_cc = float(np.sum(compute_all_clustering_coefficients(adj_mat)))/float(n)
    return avg_cc

def indices(a, func):
    return [i for (i, val) in enumerate(a) if func(val)]

# Graph Sampling Algorithms

### Node Sampling (Induced Subgraph Sampling)

In [2]:
# Takes an adjacency matrix, A, and fraction of nodes to be sampled, f.  At each 
# step, selects a node to be sampled, until f fraction of nodes have been chosen.
# Then, pick all edges induced by the set of chosen nodes
def graph_sampling_algo_uniform_node_sampling(A, f):
    n = A.shape[0]
    A_s = np.zeros(A.shape)    
    if f > 1 or f < 0:
        print "[UNS] Warning: f param not a proper fraction in uniform NS algo"
        return A
    
    # Use the unweighted adjacency matrix
    adj_mat = np.where(A != 0, 1, 0)
    num_edges_total = compute_graph_volume(adj_mat)
    num_edges_to_sample = np.ceil(f * num_edges_total)
    
    
    V = np.arange(n)
    np.random.shuffle(V)
    V_s = []
    j = 0
    num_edges_sampled = 0
    
    while j < n and num_edges_sampled < num_edges_to_sample:
        current_node = V[j]
        V_s.append(current_node)
        num_edges_sampled += np.sum(adj_mat[:, current_node])
        j += 1
        
    for node in V_s:
        A_s[:, node] = A[:, node]
        A_s[node, :] = A[node, :]
    
    return A_s
 
       

### Edge Sampling (Incident Subgraph Sampling)

In [3]:
def graph_sampling_algo_uniform_edge_sampling(A, f):
    n = A.shape[0]
    A_s = np.zeros(A.shape)
    
    if f > 1 or f < 0:
        print "[UES] Warning: f param not a proper fraction"
        return A
    
    # Use the unweighted matrix
    adj_mat = np.where(A != 0, 1, 0)
    num_edges_total = compute_graph_volume(adj_mat)
    num_edges_to_sample = np.ceil(f * num_edges_total)
    
    j = 0
    num_edges_sampled = 0
    
    # Get all the locations of unique edges
    A_triu = np.triu(A)
    E = np.nonzero(A_triu)
    E = zip(E[0], E[1])
    np.random.shuffle(E)
    
    while j < len(E) and num_edges_sampled < num_edges_to_sample:
        current_edge = E[j]
        A_s[current_edge] = A[current_edge]
        A_s[current_edge[::-1]] = A[current_edge[::-1]]
        num_edges_sampled += 1
        j += 1
        
    return A_s
    

### Metropolis-Hastings Random Walk

In [4]:
# A: possibly weighted adjacency matrix (weights WILL affect exploration,
#    but will not affect the number of edges sampled)
# f: fraction of edges to sample
# q: probability of randomly restarting at a new node
def graph_sampling_algo_metropolis_hastings_random_walk(A, f, q):
    n = A.shape[0]
    A_s = np.copy(A)
    
    if f > 1 or f < 0:
        print "[MHRW] Warning: f param not a proper fraction"
        return A
    
    if q > 1 or q < 0:
        print "[MHRW] Warning: q param is not a proper probability"
        return A
    
    adj_mat = np.where(A != 0, 1, 0)
    num_edges_total = compute_graph_volume(adj_mat)
    num_edges_to_sample = np.ceil(f * num_edges_total)
        
    # Select a starting node
    V = np.arange(n)
    np.random.shuffle(V)
    current_node = V[0]
    V_s = [current_node]
    num_edges_sampled = np.sum(adj_mat[:, current_node])
    A_s[:, current_node] = 0
    A_s[current_node, :] = 0
    
    while num_edges_sampled < num_edges_to_sample:
        
        # Determine whether to jump to a new random node or sample a neighbor
        r_decider = np.random.uniform()
        if r_decider < q:
            # Choose random restart node
            inverse_V_s = list(set(V) - set(V_s))
            np.random.shuffle(inverse_V_s)
            current_node = inverse_V_s[0]
            V_s.append(current_node)
            num_edges_sampled += np.sum(adj_mat[:, current_node])
            A_s[:, current_node] = 0
            A_s[current_node, :] = 0
            
        else:
            # Select one of the neighbors of the current node based on proportion
            # of edge weight over weighted degree
            nbrhd = get_neighborhood_set_of_node(adj_mat, current_node)
            
            # If the neighborhood is empty, force a jump to a new node
            if not nbrhd:
                # Choose random restart node
                inverse_V_s = list(set(V) - set(V_s))
                np.random.shuffle(inverse_V_s)
                current_node = inverse_V_s[0]
                V_s.append(current_node)
                num_edges_sampled += np.sum(adj_mat[:, current_node])
                A_s[:, current_node] = 0
                A_s[current_node, :] = 0
                continue
            
            degree = np.sum(A[:, current_node])
            r_decider = np.random.uniform() * degree
            cumulative_deg = 0
            for nbr_node in nbrhd:
                cumulative_deg += A[current_node, nbr_node]
                if cumulative_deg > r_decider:
                    current_node = nbr_node
                    V_s.append(current_node)
                    num_edges_sampled += np.sum(adj_mat[:, current_node])
                    A_s[:, current_node] = 0
                    A_s[current_node, :] = 0
                    break
        
    # Create A_s by inverting
    A_s = np.copy(A) - A_s
                        
    return A_s

### Frontier Sampling

In [5]:
# A: possibly weighted adjacency matrix (weights WILL affect exploration,
#    but will not affect the number of edges sampled)
# f: fraction of edges to sample
# d: deimension of the multi-dimensional random walk
def graph_sampling_algo_frontier_sampling(A, f, d):
    n = A.shape[0]
    A_s = np.zeros(A.shape)
    
    if f > 1 or f < 0:
        print "[FS] Warning: f param not a proper fraction"
        return A
    
    if d < 0:
        print "[FS] Warning: d param must be a positive integer"
        return A
        
    adj_mat = np.where(A != 0, 1, 0)
    num_edges_total = compute_graph_volume(adj_mat)
    num_edges_to_sample = np.ceil(f * num_edges_total)
    
    # Allocate list of random walker current nodes
    L = np.random.randint(0, n, d)
    
    # Keep a list of node degrees associated with the nodes in L
    degrees = np.zeros((d, 1))
    for j in range(0, d):
        degrees[j] += np.sum(A[L[j], :])
    
    num_edges_sampled = 0
    step = 0
    while num_edges_sampled < num_edges_to_sample and step <= n**3:
        # Compute sum of degrees of current nodes
        tot_deg = np.sum(degrees)
        
        # Prepare bin edges
        bin_edges = np.zeros((d+1, 1))
        for ii in range(0, d):
            bin_edges[ii+1] = bin_edges[ii] + float(degrees[ii])/float(tot_deg)
        if bin_edges[d] != 1:
            bin_edges[d] = 1.0
        
        # Use bins to choose node according to proabilities weighted by node deg
        r = np.random.uniform()
        u_idx_in_L = np.max(np.nonzero(np.where(bin_edges <= r, 1, 0)))
        u = L[u_idx_in_L]
        
        # Get neighborhood of current active random walker
        nbrhd = get_neighborhood_set_of_node(adj_mat, u)
        # Select one of the neighbors, uniformly at random
        n_neighbors = len(nbrhd)
        v = nbrhd[np.random.randint(0, n_neighbors)]
        
        # Update random walker list and degrees
        L[u_idx_in_L] = v
        degrees[u_idx_in_L] = np.sum(A[v, :])
        
        # Add the selected edge to A_s if not already present
        if A_s[u, v] == 0:
            A_s[u, v] = A[u, v]
            A_s[v, u] = A[v, u]
            num_edges_sampled += 1
        
        step += 1
        
    return A_s
    

### Snowball Expansion Sampling

In [6]:
def find_neighb(A, S, node_idx):
    nn = A.shape[0]
    
    nbrS = []
    nbrSBAR = []
    for ii in range(0, nn):
        if A[node_idx, ii] != 0:
            if not ii in S:
                nbrSBAR.append(ii)
            else:
                nbrS.append(ii)
    
    return nbrS, nbrSBAR
    

def find_boundary_nodes(A, S):
    nn = A.shape[0]
    magS = len(S)
    
    nbrS = []
    nbrSBAR = []
    for ii in range(0, magS):
        for jj in range(0, nn):
            if A[S[ii], jj] != 0:
                if not jj in S:
                    nbrSBAR.append(jj)
                else:
                    nbrS.append(jj)
                    
    return nbrS, nbrSBAR

def choose_optimal_node(A, S):
    nS, nSBAR = find_boundary_nodes(A, S)
    
    nSBAR_size = len(nSBAR)
    
    if nSBAR_size == 0:
        print "[SES] There are no neighbors to S outside of S"
        return -1
    
    XSN_factor = np.zeros((nSBAR_size, 1))
    # For each node just outside S...
    for i in range(0, nSBAR_size):
        # Find its neighbors
        t_neighb, t_neighbSBAR = find_neighb(A, S, nSBAR[i])
        XSN_factor[i] = len(t_neighbSBAR)
        
    possible_nodes = indices(XSN_factor, lambda x: x == x.max())
    p_sz = len(possible_nodes)
    
    return nSBAR[possible_nodes[np.random.randint(0, p_sz)]]


# A: possibly weighted adjacency matrix (weights WILL affect exploration,
#    but will not affect the number of edges sampled)
# f: fraction of edges to sample
def graph_sampling_algo_snowball_expansion_sampling(A, f):
    n = A.shape[0]
    A_s = np.copy(A)
    
    if f > 1 or f < 0:
        print "[SES] Warning: f param not a proper fraction"
        return A
    
    adj_mat = np.where(A != 0, 1, 0)
    num_edges_total = compute_graph_volume(adj_mat)
    num_edges_to_sample = np.ceil(f * num_edges_total)
    
    # Select starting node from discrete uniform
    node_idx = np.random.randint(0, n)
    
    # Create a list of sampled nodes
    S = [node_idx]
    
    # Keep track of number of edges sampled
    num_edges_sampled = 0
    
    while num_edges_sampled < num_edges_to_sample:
        # Choose the optimal expansion node and add it to the sample
        node_idx = choose_optimal_node(A, S)
            
        if node_idx >= 0:
            S.append(node_idx)
            num_edges_sampled += np.sum(adj_mat[:, node_idx])
            A_s[:, node_idx] = 0
            A_s[node_idx, :] = 0
        elif f == 1:
            break
        else:
            # Not a connected graph, so start somewhere else
            choices = list(set(np.arange(n)) - set(S))
            l_choices = len(choices)
            if l_choices <= 0:
                print "[SES] Error trying to restart"
                return A
            else:
                print "[SES] Restarting in new location"
                node_idx = choices[np.random.randint(0, l_choices)]
                S.append(node_idx)
                num_edges_sampled += sum(adj_mat[:, node_idx])
                A_s[:, node_idx] = 0
                A_s[node_idx, :] = 0
                
    # Invert to give the proper sampled graph
    A_s = np.copy(A) - A_s
    
    # Trim down A_s by randomly removing edges
    E_s = np.nonzero(A_s)
    E_s = zip(E_s[0], E_s[1])
    while num_edges_sampled > num_edges_to_sample + 2:
        e_to_remove = np.random.randint(0, len(E_s))
        i, j = E_s[e_to_remove]
        # Remove the edge from A_s
        A_s[i, j] = 0
        A_s[j, i] = 0
        # Remove the edge from the edge list
        del E_s[e_to_remove]
        # Decrement the number of edges sampled
        num_edges_sampled -= 1
        
    return A_s        
        

### Forest Fire Sampling

In [7]:
def burn(A, A_s, p, num_edges_sampled, num_edges_to_sample, node_idx):
    
    if num_edges_sampled <= num_edges_to_sample:
        n = A.shape[0]
        adj_mat = np.where(A != 0, 1, 0)

        burn_list = []
        for i in range(0, n):
            if A[node_idx, i] != 0 and A_s[node_idx, i] == 0:
                rndm_num = np.random.uniform()
                if rndm_num < p:
                    burn_list.append(i)

        burn_num = len(burn_list)
        # Base case: If no edges to burn, we are done
        if burn_num == 0:
            return A_s, num_edges_sampled
        # Base case: add edge to sample
        elif burn_num == 1:
            A_s[node_idx, burn_list[0]] = A[node_idx, burn_list[0]]
            A_s[burn_list[0], node_idx] = A_s[burn_list[0], node_idx]
            num_edges_sampled += 1
        # Recursive case: randomly choose which incident edge to begin with, and
        # burn each possible edge, in order
        else:
            np.random.shuffle(burn_list)
            for j in range(0, burn_num):
                A_s[node_idx, burn_list[j]] = A[node_idx, burn_list[j]]
                A_s[burn_list[j], node_idx] = A[burn_list[j], node_idx]
                num_edges_sampled += 1
                burn(A, A_s, p, num_edges_sampled,num_edges_to_sample, burn_list[j])

        return A_s, num_edges_sampled


# A: possibly weighted adjacency matrix (weights WILL affect exploration,
#    but will not affect the number of edges sampled)
# f: fraction of edges to sample
# p: probability of adding each single incident edge to the sample
def graph_sampling_algo_forest_fire_sampling(A, f, p):
    n = A.shape[0]
    A_s = np.zeros(A.shape)
    
    if f > 1 or f < 0:
        print "[FFS] Warning: f param not a proper fraction"
        return A
    
    adj_mat = np.where(A != 0, 1, 0)
    num_edges_total = compute_graph_volume(adj_mat)
    num_edges_to_sample = np.ceil(f * num_edges_total)
    
    # Restart burn process as often as necessary to reach sampling budget
    num_edges_sampled = 0
    while num_edges_sampled <= num_edges_to_sample:
        # Select a random starting node and burn from there
        node_idx = np.random.randint(0, n)
        # Recursively burn links
        A_s, num_edges_sampled = burn(A, A_s, p, num_edges_sampled, num_edges_to_sample, node_idx)
        
    return A_s
    

# Tests

In [8]:
z = np.array([[0, 2, 3, 4],
              [2, 0, 3, 4],
              [3, 3, 0, 4],
              [4, 4, 4, 0]])
z

array([[0, 2, 3, 4],
       [2, 0, 3, 4],
       [3, 3, 0, 4],
       [4, 4, 4, 0]])

In [9]:
graph_sampling_algo_uniform_node_sampling(z, 0.5)

array([[0., 0., 0., 4.],
       [0., 0., 0., 4.],
       [0., 0., 0., 4.],
       [4., 4., 4., 0.]])

In [10]:
graph_sampling_algo_uniform_edge_sampling(z, 0.5)

array([[0., 0., 0., 0.],
       [0., 0., 3., 4.],
       [0., 3., 0., 4.],
       [0., 4., 4., 0.]])

In [11]:
graph_sampling_algo_metropolis_hastings_random_walk(z, 0.5, 0.0)

array([[0, 0, 3, 0],
       [0, 0, 3, 0],
       [3, 3, 0, 4],
       [0, 0, 4, 0]])

In [12]:
graph_sampling_algo_frontier_sampling(z, 0.5, 2)

array([[0., 2., 0., 4.],
       [2., 0., 0., 4.],
       [0., 0., 0., 0.],
       [4., 4., 0., 0.]])

In [13]:
graph_sampling_algo_snowball_expansion_sampling(z, 0.5)

array([[0, 2, 0, 0],
       [2, 0, 3, 4],
       [0, 3, 0, 0],
       [0, 4, 0, 0]])

In [14]:
graph_sampling_algo_forest_fire_sampling(z, 0.5, 0.5)

array([[0., 2., 3., 4.],
       [2., 0., 3., 4.],
       [3., 3., 0., 4.],
       [4., 4., 4., 0.]])